In [ ]:
# Install libraries
!pip install fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.3 MB/s eta 0:00:00


In [ ]:
# Import libaries
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz

In [ ]:
# Read in 2024 ResearchOutputs.xlsx file
try:
  outputs_org = pd.read_excel('ResearchOutputs.xlsx', sheet_name='Sheet1')
except FileNotFoundError:
  print("Error: The file 'ResearchOutputs.xlsx' was not found.")
except Exception as e:
  print(f"An error occurred while reading the Excel file: {str(e)}")
outputs_org

,ProjectID,ProjectStatus,ProjectTitle,ProjectRDC,ProjectStartYear,ProjectEndYear,ProjectPI,OutputTitle,OutputBiblio,OutputType,OutputStatus,OutputVenue,OutputYear,OutputMonth,OutputVolume,OutputNumber,OutputPages
0,5,Completed,The Impact of Foreign Trade on the U.S. Economy,Boston,2001,2004.0,J Bradford Jensen,"Entry, Expansion, and Intensity in the U.S. Ex...","Bernard, A.B. and Jensen, J.B. (2004), Entry, ...",JA,PB,Review of International Economics,2004.0,NaN,12,4,662-675
1,5,Completed,The Impact of Foreign Trade on the U.S. Economy,Boston,2001,2004.0,J Bradford Jensen,Product Choice and Product Switching,"Bernard, Andrew B., Stephen Redding and Peter ...",WP,PB,NBER,2003.0,NaN,NaN,9789,NaN
2,5,Completed,The Impact of Foreign Trade on the U.S. Economy,Boston,2001,2004.0,J Bradford Jensen,Plants and Productivity in International Trade,"Bernard, Andrew, B., Jonathan Eaton, J. Bradfo...",JA,PB,American Economic Review,2003.0,NaN,93,4,1268-1290
3,5,Completed,The Impact of Foreign Trade on the U.S. Economy,Boston,2001,2004.0,J Bradford Jensen,Why Some Firms Export,"Bernard, Andrew B., J. Bradford Jensen. Review...",JA,PB,Review of Economics and Statistics,2004.0,NaN,86,2,561-569
4,5,Completed,The Impact of Foreign Trade on the U.S. Economy,Boston,2001,2004.0,J Bradford Jensen,The Deaths of Manufacturing Plants,"Bernard, Andrew B. and J. Bradford Jensen. “Th...",WP,PB,NBER,2002.0,NaN,NaN,9026,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,2797,Active,"Parental Wealth, Post-Secondary Human Capital ...",Wisconsin,2022,NaN,Jeremy A Kirk,The Impact of Parental Resources on Human Capi...,"Jeremy Kirk, 2024. ""The Impact of Parental Res...",WP,PB,Center for Economic Studies,2024.0,June,NaN,NaN,NaN
1731,2857,Active,"Common Ownership, Labor Market Outcomes, and C...",Philadelphia,2023,NaN,Xin Dai,Common Ownership in Labor Markets,"Azar, José and Dai, Xin and Qiu, Yue and Sojou...",MI,UP,NaN,2023.0,October,NaN,NaN,NaN
1732,2862,Active,Labor Union Membership and the Baby Boom,Chicago,2023,NaN,Henry Downes,Did Organized Labor Induce Labor? Unionization...,"Downes, Henry. Did Organized Labor Induce Labo...",MI,UP,NaN,2024.0,June,NaN,NaN,NaN
1733,2896,Active,Military Service and Immigrants' Assimilation:...,Philadelphia,2023,NaN,Melissa Desfor,Military Service and Immigrants’ Integration: ...,"Zhang, Nan, and Melissa M. Lee. 2024. “Militar...",MI,UP,NaN,2024.0,NaN,NaN,NaN,NaN


In [ ]:
# Check for missing values
outputs_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1735 entries, 0 to 1734
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ProjectID         1735 non-null   int64  
 1   ProjectStatus     1735 non-null   object 
 2   ProjectTitle      1735 non-null   object 
 3   ProjectRDC        1735 non-null   object 
 4   ProjectStartYear  1735 non-null   int64  
 5   ProjectEndYear    1355 non-null   float64
 6   ProjectPI         1735 non-null   object 
 7   OutputTitle       1734 non-null   object 
 8   OutputBiblio      1735 non-null   object 
 9   OutputType        1735 non-null   object 
 10  OutputStatus      1734 non-null   object 
 11  OutputVenue       1488 non-null   object 
 12  OutputYear        1731 non-null   float64
 13  OutputMonth       353 non-null    object 
 14  OutputVolume      462 non-null    object 
 15  OutputNumber      842 non-null    object 
 16  OutputPages       387 non-null    object 


In [ ]:
# Visualize the null value columns
outputs_org[outputs_org["OutputTitle"].isna()]

,ProjectID,ProjectStatus,ProjectTitle,ProjectRDC,ProjectStartYear,ProjectEndYear,ProjectPI,OutputTitle,OutputBiblio,OutputType,OutputStatus,OutputVenue,OutputYear,OutputMonth,OutputVolume,OutputNumber,OutputPages
1604,2229,Active,"The Impact of Trade on Consumption, Earnings, ...",Utah,2022,NaN,Jiashuo Feng,NaN,"Borusyak, Kirill and Xavier Jaravel. Are Trade...",JA,PB,Journal of International Economics,2024.0,July,NaN,NaN,NaN


In [ ]:
# Replace reasearch outputs without OutputTitle with ProjectTitle as title
outputs_org.loc[outputs_org["OutputTitle"].isna(), "OutputTitle"] = outputs_org.loc[outputs_org["OutputTitle"].isna(), "ProjectTitle"]

In [ ]:
# Visualize the null value columns
outputs_org[outputs_org["OutputYear"].isna()]

,ProjectID,ProjectStatus,ProjectTitle,ProjectRDC,ProjectStartYear,ProjectEndYear,ProjectPI,OutputTitle,OutputBiblio,OutputType,OutputStatus,OutputVenue,OutputYear,OutputMonth,OutputVolume,OutputNumber,OutputPages
26,81,Completed,Comparison of the Distributions of Production ...,Washington,2001,2001.0,Gale A Boyd,Standard Errors for a Consistent Estimator of ...,"Boyd, Gale A. and Julian Silk ""Standard Errors...",MI,UP,NaN,NaN,NaN,NaN,NaN,NaN
1070,1284,Completed,"The Long-Run Determinants of Social, Demograph...",Michigan,2015,2020.0,Martha J Bailey,The Long-Term Health and Economic Benefits of ...,"Bailey, Martha J. and Olga Malkova. ""The Long-...",MI,UP,NaN,NaN,NaN,NaN,NaN,NaN
1072,1284,Completed,"The Long-Run Determinants of Social, Demograph...",Michigan,2015,2020.0,Martha J Bailey,The Long-Term Health and Economic Benefits of ...,"Bailey, Martha J., Valentina Duque, and Andre...",MI,UP,NaN,NaN,NaN,NaN,NaN,NaN
1075,1284,Completed,"The Long-Run Determinants of Social, Demograph...",Michigan,2015,2020.0,Martha J Bailey,The War on Poverty and Economic Mobility,"Bailey, Martha J. and Nicolas Duquette. ""The W...",MI,UP,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Replace reasearch outputs without OutputYear with ProjectEndYear as year
outputs_org.loc[outputs_org["OutputYear"].isna(), "OutputYear"] = outputs_org.loc[outputs_org["OutputYear"].isna(), "ProjectEndYear"]

In [ ]:
# Only keep the columns needed
outputs_org = outputs_org[["OutputTitle", "OutputYear", "ProjectRDC", "ProjectPI"]].copy()

In [ ]:
# Change the column names
outputs_org.columns = ["Title", "Year", "Agency", "PI"]
# Change the column type of Year column to be int
outputs_org["Year"] = outputs_org["Year"].astype(int)
# Strip other text columns
outputs_org["Title"] = outputs_org["Title"].str.strip()
outputs_org["Agency"] = outputs_org["Agency"].str.strip()
outputs_org["PI"] = outputs_org["PI"].str.strip()

In [ ]:
outputs_org

,Title,Year,Agency,PI
0,"Entry, Expansion, and Intensity in the U.S. Ex...",2004,Boston,J Bradford Jensen
1,Product Choice and Product Switching,2003,Boston,J Bradford Jensen
2,Plants and Productivity in International Trade,2003,Boston,J Bradford Jensen
3,Why Some Firms Export,2004,Boston,J Bradford Jensen
4,The Deaths of Manufacturing Plants,2002,Boston,J Bradford Jensen
...,...,...,...,...
1730,The Impact of Parental Resources on Human Capi...,2024,Wisconsin,Jeremy A Kirk
1731,Common Ownership in Labor Markets,2023,Philadelphia,Xin Dai
1732,Did Organized Labor Induce Labor? Unionization...,2024,Chicago,Henry Downes
1733,Military Service and Immigrants’ Integration: ...,2024,Philadelphia,Melissa Desfor


In [ ]:
# Read API retrieved researcch outputs from csv file
try:
  outputs = pd.read_csv("openalex_researcher_datasets_matches.csv")
except FileNotFoundError:
  print("Error: The file 'openalex_researcher_datasets_matches.csv' was not found.")
except Exception as e:
  print(f"An error occurred while reading the Excel file: {str(e)}")
outputs

,title,doi,abstract,year,publication_date,cited_by_count,authors,affiliations,topics,source_display_name,type_crossref,researcher,author_id,queried_dataset_terms,matched_dataset_terms,location
0,The Bridge between the LexisNexis Directory of...,NaN,The purpose of this matching exercise is to co...,2015,1/1/2015,0,Aaron Flaaen; Yue Maggie Zhou,NaN,Cooperative Studies and Economics,NaN,article,Aaron B Flaaen,https://openalex.org/A5081886594,Survey of Plant Capacity Utilization; Foreign ...,Census Bureau,Michigan
1,Vulnerable Jobs and the Wage Effects of Import...,https://doi.org/10.1111/irel.12240,Do job characteristics modulate the relationsh...,2019,6/10/2019,1,Abigail M. Cooke; Thomas Kemeny; David L. Rigby,"University at Buffalo, State University of New...",Employment and Welfare Studies; Labor market d...,Industrial Relations A Journal of Economy and ...,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
2,Task Trade and the Wage Effects of Import Comp...,https://doi.org/10.2139/ssrn.2714864,Do job characteristics modulate the relationsh...,2016,1/1/2016,0,Abigail M. Cooke; Thomas Kemney; David L. Rigby,"University at Buffalo, State University of New...",Global trade and economics,SSRN Electronic Journal,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
3,Task Trade and the Wage Effects of Import Comp...,NaN,Do job characteristics modulate the relationsh...,2016,1/1/2016,0,Abigail M. Cooke; Thomas Kemeny; David L. Rigby,NaN,"Global trade and economics; Global trade, sust...",RePEc: Research Papers in Economics,preprint,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
4,Impacts of Trade on Wage Inequality across the...,NaN,The research presented in this dissertation ex...,2014,1/1/2014,0,Abigail M. Cooke,NaN,Regional Economic and Spatial Analysis,NaN,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656,Dispersion in Dispersion: Measuring Establishm...,https://doi.org/10.2139/ssrn.3870190,We describe new experimental productivity stat...,2021,1/1/2021,9,Cindy Cunningham; Lucia Foster; Cheryl Grim; J...,Bureau of Labor Statistics; United States Cens...,Regional Economics and Spatial Analysis; Econo...,SSRN Electronic Journal,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board
1657,Dispersion in Dispersion: Measuring Establishm...,https://doi.org/10.1111/roiw.12616,Abstract We describe new experimental producti...,2022,9/26/2022,12,Cindy Cunningham; Lucia Foster; Cheryl Grim; J...,Bureau of Labor Statistics; United States Cens...,Firm Innovation and Growth; Global trade and e...,Review of Income and Wealth,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board
1658,Opening the Black Box: Task and Skill Mix and ...,https://doi.org/10.3386/w30620,An important gap in most empirical studies of ...,2022,11/1/2022,1,G. Jacob Blackwood; Cindy Cunningham; Matthew ...,NaN,Labor market dynamics and wage inequality; Fir...,NaN,report,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board
1659,Opening the Black Box: Task and Skill Mix and ...,https://doi.org/10.2139/ssrn.4234908,An important gap in most empirical studies of ...,2022,1/1/2022,0,G. Jacob Blackwood; Cindy Cunningham; Matt Dey...,Amherst College; Bureau of Labor Statistics; U...,Labor market dynamics and wage inequality,SSRN Electronic Journal,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board


In [ ]:
outputs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1661 entries, 0 to 1660
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   title                  1657 non-null   object
 1   doi                    1123 non-null   object
 2   abstract               1661 non-null   object
 3   year                   1661 non-null   int64 
 4   publication_date       1661 non-null   object
 5   cited_by_count         1661 non-null   int64 
 6   authors                1661 non-null   object
 7   affiliations           1130 non-null   object
 8   topics                 1651 non-null   object
 9   source_display_name    1361 non-null   object
 10  type_crossref          1661 non-null   object
 11  researcher             1661 non-null   object
 12  author_id              1661 non-null   object
 13  queried_dataset_terms  1661 non-null   object
 14  matched_dataset_terms  1661 non-null   object
 15  location             

In [ ]:
outputs.iloc[1,:]["source_display_name"]

'Industrial Relations A Journal of Economy and Society'

In [ ]:
# Convert publication_date column to datetime
outputs["publication_date"] = pd.to_datetime(outputs["publication_date"])
# Split the authors and save it in a set
outputs["authors"] = outputs["authors"].apply(lambda x: {item.strip() for item in x.split(";")})
outputs["researcher"] = outputs["researcher"].str.strip()
outputs

,title,doi,abstract,year,publication_date,cited_by_count,authors,affiliations,topics,source_display_name,type_crossref,researcher,author_id,queried_dataset_terms,matched_dataset_terms,location
0,The Bridge between the LexisNexis Directory of...,NaN,The purpose of this matching exercise is to co...,2015,2015-01-01,0,"{Aaron Flaaen, Yue Maggie Zhou}",NaN,Cooperative Studies and Economics,NaN,article,Aaron B Flaaen,https://openalex.org/A5081886594,Survey of Plant Capacity Utilization; Foreign ...,Census Bureau,Michigan
1,Vulnerable Jobs and the Wage Effects of Import...,https://doi.org/10.1111/irel.12240,Do job characteristics modulate the relationsh...,2019,2019-06-10,1,"{Abigail M. Cooke, Thomas Kemeny, David L. Rigby}","University at Buffalo, State University of New...",Employment and Welfare Studies; Labor market d...,Industrial Relations A Journal of Economy and ...,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
2,Task Trade and the Wage Effects of Import Comp...,https://doi.org/10.2139/ssrn.2714864,Do job characteristics modulate the relationsh...,2016,2016-01-01,0,"{Abigail M. Cooke, David L. Rigby, Thomas Kemney}","University at Buffalo, State University of New...",Global trade and economics,SSRN Electronic Journal,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
3,Task Trade and the Wage Effects of Import Comp...,NaN,Do job characteristics modulate the relationsh...,2016,2016-01-01,0,"{Abigail M. Cooke, Thomas Kemeny, David L. Rigby}",NaN,"Global trade and economics; Global trade, sust...",RePEc: Research Papers in Economics,preprint,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
4,Impacts of Trade on Wage Inequality across the...,NaN,The research presented in this dissertation ex...,2014,2014-01-01,0,{Abigail M. Cooke},NaN,Regional Economic and Spatial Analysis,NaN,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656,Dispersion in Dispersion: Measuring Establishm...,https://doi.org/10.2139/ssrn.3870190,We describe new experimental productivity stat...,2021,2021-01-01,9,"{Zoltán Wolf, Jay M. Stewart, Cheryl Grim, Luc...",Bureau of Labor Statistics; United States Cens...,Regional Economics and Spatial Analysis; Econo...,SSRN Electronic Journal,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board
1657,Dispersion in Dispersion: Measuring Establishm...,https://doi.org/10.1111/roiw.12616,Abstract We describe new experimental producti...,2022,2022-09-26,12,"{Zoltán Wolf, Cheryl Grim, Lucia Foster, Sabri...",Bureau of Labor Statistics; United States Cens...,Firm Innovation and Growth; Global trade and e...,Review of Income and Wealth,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board
1658,Opening the Black Box: Task and Skill Mix and ...,https://doi.org/10.3386/w30620,An important gap in most empirical studies of ...,2022,2022-11-01,1,"{Cody Tuttle, Zoltán Wolf, Matthew Dey, Cheryl...",NaN,Labor market dynamics and wage inequality; Fir...,NaN,report,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board
1659,Opening the Black Box: Task and Skill Mix and ...,https://doi.org/10.2139/ssrn.4234908,An important gap in most empirical studies of ...,2022,2022-01-01,0,"{Cody Tuttle, Zoltán Wolf, Matt Dey, Cheryl Gr...",Amherst College; Bureau of Labor Statistics; U...,Labor market dynamics and wage inequality,SSRN Electronic Journal,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bur

In [ ]:
outputs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1661 entries, 0 to 1660
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   title                  1657 non-null   object        
 1   doi                    1123 non-null   object        
 2   abstract               1661 non-null   object        
 3   year                   1661 non-null   int64         
 4   publication_date       1661 non-null   datetime64[ns]
 5   cited_by_count         1661 non-null   int64         
 6   authors                1661 non-null   object        
 7   affiliations           1130 non-null   object        
 8   topics                 1651 non-null   object        
 9   source_display_name    1361 non-null   object        
 10  type_crossref          1661 non-null   object        
 11  researcher             1661 non-null   object        
 12  author_id              1661 non-null   object        
 13  que

In [ ]:
# Add a column Unique to identify if the output is unqiue to the dataset
# (The research output does not exit in 2024 dataset)
outputs["unique"] = False

In [ ]:
# Filter unique outputs by year
# Output published outside the range of 2024 dataset is definitely unique output
outputs.loc[~outputs["year"].isin(outputs_org.Year.unique()), "unique"] = True
outputs

,title,doi,abstract,year,publication_date,cited_by_count,authors,affiliations,topics,source_display_name,type_crossref,researcher,author_id,queried_dataset_terms,matched_dataset_terms,location,unique
0,The Bridge between the LexisNexis Directory of...,NaN,The purpose of this matching exercise is to co...,2015,2015-01-01,0,"{Aaron Flaaen, Yue Maggie Zhou}",NaN,Cooperative Studies and Economics,NaN,article,Aaron B Flaaen,https://openalex.org/A5081886594,Survey of Plant Capacity Utilization; Foreign ...,Census Bureau,Michigan,False
1,Vulnerable Jobs and the Wage Effects of Import...,https://doi.org/10.1111/irel.12240,Do job characteristics modulate the relationsh...,2019,2019-06-10,1,"{Abigail M. Cooke, Thomas Kemeny, David L. Rigby}","University at Buffalo, State University of New...",Employment and Welfare Studies; Labor market d...,Industrial Relations A Journal of Economy and ...,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle,False
2,Task Trade and the Wage Effects of Import Comp...,https://doi.org/10.2139/ssrn.2714864,Do job characteristics modulate the relationsh...,2016,2016-01-01,0,"{Abigail M. Cooke, David L. Rigby, Thomas Kemney}","University at Buffalo, State University of New...",Global trade and economics,SSRN Electronic Journal,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle,False
3,Task Trade and the Wage Effects of Import Comp...,NaN,Do job characteristics modulate the relationsh...,2016,2016-01-01,0,"{Abigail M. Cooke, Thomas Kemeny, David L. Rigby}",NaN,"Global trade and economics; Global trade, sust...",RePEc: Research Papers in Economics,preprint,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle,False
4,Impacts of Trade on Wage Inequality across the...,NaN,The research presented in this dissertation ex...,2014,2014-01-01,0,{Abigail M. Cooke},NaN,Regional Economic and Spatial Analysis,NaN,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656,Dispersion in Dispersion: Measuring Establishm...,https://doi.org/10.2139/ssrn.3870190,We describe new experimental productivity stat...,2021,2021-01-01,9,"{Zoltán Wolf, Jay M. Stewart, Cheryl Grim, Luc...",Bureau of Labor Statistics; United States Cens...,Regional Economics and Spatial Analysis; Econo...,SSRN Electronic Journal,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board,False
1657,Dispersion in Dispersion: Measuring Establishm...,https://doi.org/10.1111/roiw.12616,Abstract We describe new experimental producti...,2022,2022-09-26,12,"{Zoltán Wolf, Cheryl Grim, Lucia Foster, Sabri...",Bureau of Labor Statistics; United States Cens...,Firm Innovation and Growth; Global trade and e...,Review of Income and Wealth,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board,False
1658,Opening the Black Box: Task and Skill Mix and ...,https://doi.org/10.3386/w30620,An important gap in most empirical studies of ...,2022,2022-11-01,1,"{Cody Tuttle, Zoltán Wolf, Matthew Dey, Cheryl...",NaN,Labor market dynamics and wage inequality; Fir...,NaN,report,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board,False
1659,Opening the Black Box: Task and Skill Mix and ...,https://doi.org/10.2139/ssrn.4234908,An important gap in most empirical studies of ...,2022,2022-01-01,0,"{Cody Tuttle, Zoltán Wolf, Matt Dey, Cheryl Gr...",Amherst College; Bureau of Labor Statistics; U...,Labor market dynamics and wage inequality,SSRN Electronic Journal,article,Zoltan Wolf,https://openalex.org/A5008460009,S

In [ ]:
outputs["unique"].value_counts()

,count
unique,
False,1623
True,38


In [ ]:
# Define a function to do fuzzy matching
def find_unique_records(outputs_org, outputs, threshold=90):
  """
  Find unique records in outputs where no records has the same year
  and the similarity score for the title is greater or equal to 90

  Parameters:
  - outputs_org: DataFrame of 2024 dataset
  - outputs: DataFrame of API retrieved research outputs
  - threshold: Minimum similarity score to consider as a match (0-100)

  Returns:
  - set of unique records indices
  - dictionary of best matches results if not unique
    - key: index in outputs
    - value: tuple(best match index in 2024 dataset,
      best_match_title, best_match_Score)
  """
  # Keep track of index of unique records and best match result otherwise
  unique_indices = set()
  best_matches = {}

  # Loop through all records in outputs to find the closest fuzzy match records
  for idx, row in outputs.iterrows():
    # Filter out the records in outputs_org that does not have the same year
    outputs_org_filtered = outputs_org[outputs_org["Year"] == row["year"]]
    # Keep track of the best macthed index, title, PI and similarity score
    best_match_idx = -1
    best_match_title = ""
    best_match_score = 0.0
    # Loop through all records in original 2024 dataset to find the best match
    for idx2, row2 in outputs_org_filtered.iterrows():
      # Keep track of whether two outputs has the same PI
      PI_flag = True

      # Check if PI is in authors/researchers (or similarity score is above threshold)
      if row2["PI"] not in row["authors"] and row2["PI"] != row["researcher"]:
        # Keep track of the author best match score
        best_PI_score = fuzz.token_sort_ratio(row["researcher"].lower(), row2["PI"].lower())
        # Loop through the authors list to get the best similarity score
        for author in row["authors"]:
          # Calculate author similarity score
          score_author = fuzz.token_sort_ratio(author.lower(), str(row2["PI"]).lower())
          # Replace the best_PI score if better
          if score_author > best_PI_score:
            best_PI_score = score_author
        # If the best score is above threshold, we consider it as the same PI
        # Skip this record if PI are considered the same
        if best_PI_score < threshold:
          PI_flag = False

      # Check if two outputs has the same PI, only proceed if True
      if PI_flag:
        # Get similarity score
        score = fuzz.token_sort_ratio(str(row["title"]).lower(), str(row2["Title"]).lower())
        # Replace the best match idx, title, score if this is better
        if score > best_match_score:
          best_match_idx = idx2
          best_match_title = row2["Title"]
          best_match_score = score
    # If best match is above the threshold, save the bet match result
    # Otherwise add idx to unique_indices set
    if best_match_score >= threshold:
      best_matches[idx] = (best_match_idx, best_match_title, best_match_score)
    else:
      unique_indices.add(idx)

  # Return both unique_indices and best_matches
  return unique_indices, best_matches

In [ ]:
# Filter by title fuzzy matching
# Only apply to records with years in 2024 dataset year range
unique_indices, best_matches = find_unique_records(outputs_org, outputs[outputs["unique"]==False])

In [ ]:
len(unique_indices), len(best_matches)

(1353, 270)

In [ ]:
# Set unique to be true for unique records
outputs.loc[outputs.index.isin(unique_indices), "unique"] = True
outputs["unique"].value_counts()

,count
unique,
True,1391
False,270


In [ ]:
# Add best match idx, title, score to outputs
outputs["best_match_idx"] = None
outputs["best_match_title"] = None
outputs["best_match_score"] = None

# Update the columns for records that have matches
for idx, match_info in best_matches.items():
  match_idx, match_title, match_score = match_info
  outputs.loc[idx, "best_match_idx"] = match_idx
  outputs.loc[idx, "best_match_title"] = match_title
  outputs.loc[idx, "best_match_score"] = match_score

In [ ]:
outputs.loc[outputs["unique"] == False, ["title", "best_match_idx", "best_match_title", "best_match_score"]]

,title,best_match_idx,best_match_title,best_match_score
2,Task Trade and the Wage Effects of Import Comp...,498,Task Trade and the Wage Effects of Import Comp...,100
3,Task Trade and the Wage Effects of Import Comp...,498,Task Trade and the Wage Effects of Import Comp...,100
12,The Impact of Immigration on Firms and Workers...,1250,The Impact of Immigration on Firms and Workers...,100
13,Public-Use vs. Restricted-Use: An Analysis Usi...,879,Public-Use vs. Restricted-Use: An Analysis Usi...,100
16,Public-Use vs. Restricted-Use: An Analysis Usi...,879,Public-Use vs. Restricted-Use: An Analysis Usi...,100
...,...,...,...,...
1639,Offshoring Pollution while Offshoring Production?,663,Offshoring Pollution while Offshoring Production,100
1641,Offshoring Production or Offshoring Pollution?,665,Offshoring Pollution while Offshoring Production,90
1642,Are Mutual Fund Managers Paid for Performance?...,1500,Are Mutual Fund Managers Paid for Performance?...,100
1647,Spatial organization of firms: the decision to...,72,Spatial Organization of Firms: The decision to...,94


In [ ]:
unique_records = outputs.loc[outputs["unique"] == True,
 ['title', 'doi', 'abstract', 'year', 'publication_date',
  'cited_by_count', 'authors', 'affiliations', 'topics',
  'source_display_name', 'type_crossref', 'researcher', 'author_id',
  'queried_dataset_terms', 'matched_dataset_terms', 'location']]
unique_records

,title,doi,abstract,year,publication_date,cited_by_count,authors,affiliations,topics,source_display_name,type_crossref,researcher,author_id,queried_dataset_terms,matched_dataset_terms,location
0,The Bridge between the LexisNexis Directory of...,NaN,The purpose of this matching exercise is to co...,2015,2015-01-01,0,"{Aaron Flaaen, Yue Maggie Zhou}",NaN,Cooperative Studies and Economics,NaN,article,Aaron B Flaaen,https://openalex.org/A5081886594,Survey of Plant Capacity Utilization; Foreign ...,Census Bureau,Michigan
1,Vulnerable Jobs and the Wage Effects of Import...,https://doi.org/10.1111/irel.12240,Do job characteristics modulate the relationsh...,2019,2019-06-10,1,"{Abigail M. Cooke, Thomas Kemeny, David L. Rigby}","University at Buffalo, State University of New...",Employment and Welfare Studies; Labor market d...,Industrial Relations A Journal of Economy and ...,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
4,Impacts of Trade on Wage Inequality across the...,NaN,The research presented in this dissertation ex...,2014,2014-01-01,0,{Abigail M. Cooke},NaN,Regional Economic and Spatial Analysis,NaN,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
5,Impacts of Trade on Wage Inequality across the...,NaN,"Author(s): Cooke, Abigail Montague | Advisor(s...",2014,2014-01-01,0,{Abigail M. Cooke},NaN,Global trade and economics; Regional Economic ...,NaN,article,Abigail M Cooke,https://openalex.org/A5072101814,LEHD Geocoded Address List (GAL) - 2008; Curre...,Census Bureau,Triangle
6,Coverage and Agreement of Administrative Recor...,https://doi.org/10.31235/osf.io/ktmz9,The U.S. Census Bureau is researching possible...,2017,2017-01-11,5,"{Sonya Rastogi, Renuka Bhaskar, Adela Luque, J...",NaN,"Health disparities and outcomes; Gender, Labor...",NaN,preprint,Adela Luque,https://openalex.org/A5110302993,Decennial Census; Census Bureau,Decennial Census; Census Bureau,Washington
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656,Dispersion in Dispersion: Measuring Establishm...,https://doi.org/10.2139/ssrn.3870190,We describe new experimental productivity stat...,2021,2021-01-01,9,"{Zoltán Wolf, Jay M. Stewart, Cheryl Grim, Luc...",Bureau of Labor Statistics; United States Cens...,Regional Economics and Spatial Analysis; Econo...,SSRN Electronic Journal,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board
1657,Dispersion in Dispersion: Measuring Establishm...,https://doi.org/10.1111/roiw.12616,Abstract We describe new experimental producti...,2022,2022-09-26,12,"{Zoltán Wolf, Cheryl Grim, Lucia Foster, Sabri...",Bureau of Labor Statistics; United States Cens...,Firm Innovation and Growth; Global trade and e...,Review of Income and Wealth,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board
1658,Opening the Black Box: Task and Skill Mix and ...,https://doi.org/10.3386/w30620,An important gap in most empirical studies of ...,2022,2022-11-01,1,"{Cody Tuttle, Zoltán Wolf, Matthew Dey, Cheryl...",NaN,Labor market dynamics and wage inequality; Fir...,NaN,report,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board
1659,Opening the Black Box: Task and Skill Mix and ...,https://doi.org/10.2139/ssrn.4234908,An important gap in most empirical studies of ...,2022,2022-01-01,0,"{Cody Tuttle, Zoltán Wolf, Matt Dey, Cheryl Gr...",Amherst College; Bureau of Labor Statistics; U...,Labor market dynamics and wage inequality,SSRN Electronic Journal,article,Zoltan Wolf,https://openalex.org/A5008460009,Survey of Plant Capacity Utilization; Service ...,Census Bureau,Fed Board


In [ ]:
unique_records.to_csv("unique_outputs_test2.csv", index=False)

In [1]:
1105+53

1158